In [12]:
# convolution_runner.py
from pynq import Overlay, allocate
import numpy as np
from PIL import Image
import os
import time

# Constants
IMG_WIDTH = 128
IMG_HEIGHT = 128
INPUT_DIR = "images"
OUTPUT_DIR = "results"

print("Loading Overlay.")
overlay = Overlay("Convolution.xsa")
overlay.download()
print("Overlay Loaded.")

conv = overlay.convolution_0
os.makedirs(OUTPUT_DIR, exist_ok=True)

for i in range(1, 11):
    img_path = f"{INPUT_DIR}/input{i}.png"
    out_path = f"{OUTPUT_DIR}/output{i}.png"

    print(f"\nProcessing {img_path} -> {out_path}")

    # Load and preprocess image
    img = Image.open(img_path).convert("L").resize((IMG_WIDTH, IMG_HEIGHT))
    img_np = np.array(img, dtype=np.uint8)

    # Allocate buffers
    in_buf = allocate(shape=(IMG_HEIGHT, IMG_WIDTH), dtype=np.uint8)
    out_buf = allocate(shape=(IMG_HEIGHT, IMG_WIDTH), dtype=np.uint8)

    # Copy image data
    np.copyto(in_buf, img_np)

    # Write 64-bit physical addresses to 32-bit register space
    in_addr = int(in_buf.physical_address)
    out_addr = int(out_buf.physical_address)

    conv.write(0x10, in_addr & 0xFFFFFFFF)       # IN_IMG lower
    conv.write(0x14, (in_addr >> 32) & 0xFFFFFFFF)  # IN_IMG upper
    conv.write(0x1C, out_addr & 0xFFFFFFFF)      # OUT_IMG lower
    conv.write(0x20, (out_addr >> 32) & 0xFFFFFFFF) # OUT_IMG upper

    # Start IP
    conv.write(0x00, 0x01)

    # Wait for IP completion
    while (conv.read(0x00) & 0x2) == 0:
        pass

    # Debug output
    print("Center pixel:", out_buf[64, 64])
    print("Sum of output:", np.sum(out_buf))

    # Save result
    out_img = Image.fromarray(out_buf).convert("L")
    out_img.save(out_path)
    print(f"Saved {out_path}")


Loading Overlay.
Overlay Loaded.

Processing images/input1.png -> results/output1.png
Center pixel: 0
Sum of output: 384067
Saved results/output1.png

Processing images/input2.png -> results/output2.png
Center pixel: 16
Sum of output: 401795
Saved results/output2.png

Processing images/input3.png -> results/output3.png
Center pixel: 0
Sum of output: 558022
Saved results/output3.png

Processing images/input4.png -> results/output4.png
Center pixel: 0
Sum of output: 528924
Saved results/output4.png

Processing images/input5.png -> results/output5.png
Center pixel: 0
Sum of output: 315742
Saved results/output5.png

Processing images/input6.png -> results/output6.png
Center pixel: 0
Sum of output: 485385
Saved results/output6.png

Processing images/input7.png -> results/output7.png
Center pixel: 0
Sum of output: 312658
Saved results/output7.png

Processing images/input8.png -> results/output8.png
Center pixel: 255
Sum of output: 141380
Saved results/output8.png

Processing images/input9.pn